Environment name: `r_env_corr`

In [55]:
R.version.string


[1] "R version 4.4.2 (2024-10-31)"

In [56]:
# BiocManager::install("EnsDb.Hsapiens.v86")
# BiocManager::install("BSgenome.Hsapiens.UCSC.hg38")
# devtools::install_github("jbergenstrahle/STUtility")
# remotes::install_github('satijalab/seurat-wrappers')
# devtools::install_github('cole-trapnell-lab/monocle3')
# devtools::install_github("GreenleafLab/ArchR", ref="master", repos = BiocManager::repositories())

In [57]:
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("GenomeInfoDb", "EnsDb.Hsapiens.v86", "BSgenome.Hsapiens.UCSC.hg38", "DropletUtils"), force=TRUE)

# install.packages(c("harmony", "gprofiler2", "loomR", "SeuratWrappers", "monocle3", "ArchR", "STutility", "parallel", "ica", "spdep", "jpeg", "ggpubr", "gplots"))

# library(GenomeInfoDb)
# library(EnsDb.Hsapiens.v86)
# library(BSgenome.Hsapiens.UCSC.hg38)
# library(Signac)
# library(Seurat)
# library(DropletUtils)
# library(harmony)
# library(ggplot2)
# library(dplyr)

In [58]:
setwd(dir = "/disk2/user/gabgam/the_project/5_integration_and_correlation/")

In [59]:
# source("/disk2/user/gabgam/work/gigi_env/the_project/data/spatial_atac/github_folder/code/source_satac.R")
source("./utils/source_satac.R")

Setting default genome to Hg38.



In [60]:
# table <- list()

# PATH_TO_sATAC_DATA_FOLDER <- "../data/spatial_atac/"
# sATAC_SAMPLE_NAME <- "220327_C1"

# # create a table containing paths to raw and meta data (i.e., output from cellranger + spatial info)
# table <- list(
#         singlecell = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", sATAC_SAMPLE_NAME, "/singlecell.csv"),
#         fragments = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", sATAC_SAMPLE_NAME, "/fragments.tsv.gz"),
#         tissue_paths = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", sATAC_SAMPLE_NAME, "_tissue.tsv"),
#         spotfiles = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", sATAC_SAMPLE_NAME, "_tissue.csv")
#         )

# # as.data.frame(table)
# table

In [61]:
# set paths to raw data and metadata
dirs <- list.dirs("../data/spatial_atac/outs/", recursive = F, full.names = F)
table <- list()

PATH_TO_sATAC_DATA_FOLDER <- "../data/spatial_atac/"
sATAC_SAMPLE_NAME <- "220327_C1"
# dirs <- list("220327_C1")

# create a table containing paths to raw and meta data (i.e., output from cellranger + spatial info)
for(i in dirs){
  table[[i]] <- list(samples = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", i, "/raw_peak_bc_matrix.h5"),
                  singlecell = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", i, "/singlecell.csv"),
                  fragments = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", i, "/fragments.tsv.gz"),
                  tissue_paths = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", i, "_tissue.tsv"),
                  spotfiles = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", i, "_tissue.csv"))
}

infoTable <- do.call("rbind", table) %>% as.data.frame()
#infoTable <- as.data.frame(table, stringsAsFactors = FALSE)

object <- list(md = list(),
               frag = list(),
               counts = list())

# build new matrices with new peak set
# peaks obtained from https://www.science.org/doi/10.1126/science.aav1898
# load peaks and create genomicranges object
BRCA_PEAK_CALLS <- paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/BRCA_peakCalls.csv")

gr <- read.csv(
  file = BRCA_PEAK_CALLS,
  col.names = c("chr", "start", "end"),
  sep = ";"
) %>% makeGRangesFromDataFrame()

# create fragment objects and count matrices for each section separately
for(i in seq_along(dirs)){
  folder = paste0(PATH_TO_sATAC_DATA_FOLDER, "combined/")
  # load metadata
  object$md[[i]] <- read.table(
    file = infoTable$singlecell[[i]],
    stringsAsFactors = FALSE,
    sep = ",",
    header = TRUE,
    row.names = 1
  )[-1, ] # remove the first row (NO_BARCODE)
  
  # create fragment objects
  object$frag[[i]] <- CreateFragmentObject(
    path = infoTable$fragments[[i]],
    cells = rownames(object$md[[i]])
  )
  
  if (file.exists(paste0(folder, "brca_peak_bc_matrix.h5"))) {
    message("File exists.")
    object$counts[[i]] <- Read10X_h5(paste0(folder, "brca_peak_bc_matrix.h5"))
    
  } else {
    
    # make count matrix
    object$counts[[i]] <- FeatureMatrix(
      fragments = object$frag[[i]],
      features = gr,
      cells = rownames(object$md[[i]])
    )
    
    write10xCounts(path = paste0(folder, "brca_peak_bc_matrix.h5"),
                   x = object$counts[[i]],
                   type = "HDF5") # save new count matrix
  }
  
}


Computing hash

File exists.

Computing hash

File exists.

Computing hash

File exists.



In [62]:
# save unmerged objects and metadata
saveRDS(object, "output/sATAC_all_samples_preprocessing/brca_unmerged.rds")

building the correct tissue_list file.

In [63]:
# make tissue position files
for(i in seq_along(infoTable$tissue_paths)){
  spotfile <- read.table(infoTable$tissue_paths[[i]], sep = "\t", header = T)
  
  spotfile_tissue <- cbind(
    spotfile$row, #array_row
    spotfile$col, #array_col
    spotfile$x, #pxl_row_in_fullres
    spotfile$y #pxl_col_in_fullres
  ) %>% as.data.frame()
  tissue <- rep(1, nrow(spotfile_tissue))
  spotfile_tissue <- cbind(tissue, spotfile_tissue)
  rownames(spotfile_tissue) <- paste0(spotfile$barcode, "-1")
  
  # save new tissue position file with the correct columns for creating STutility object as .csv files
  write.csv(spotfile_tissue, 
            paste0(strsplit(infoTable$tissue_paths[[i]], "tsv"), "csv"), 
            sep = ",")
  
}

Warning message in write.csv(spotfile_tissue, paste0(strsplit(infoTable$tissue_paths[[i]], :
"attempt to set 'sep' ignored"


Warning message in write.csv(spotfile_tissue, paste0(strsplit(infoTable$tissue_paths[[i]], :
"attempt to set 'sep' ignored"
Warning message in write.csv(spotfile_tissue, paste0(strsplit(infoTable$tissue_paths[[i]], :
"attempt to set 'sep' ignored"


loading the files just created and 

In [64]:
# load spotfiles
tissue_md <- list()
object$mtx <- list()

for(i in seq_along(infoTable$spotfiles)){
  spotfile <- read.csv(infoTable$spotfiles[[i]],
                       col.names = c("barcode", "tissue", "y", "x", "pixel_y", "pixel_x")
  )
  
  # filter count matrix to retain only spots overlaying tissue
  mtx <- object$counts[[i]] # barcodes extracted from here
  mtx <- mtx[,colnames(mtx) %in% spotfile$barcode] 
  
  object$mtx[[i]] <- mtx
  
  #save spot info in a list
  tissue_md[[i]] <- spotfile
}


In [72]:
signac_object <- list()

# create signac objects for each section and run normalization and dimensionality reduction
for(i in seq_along(infoTable$spotfiles)){
  assay <- CreateChromatinAssay(object$mtx[[i]], 
                                fragments = object$frag[[i]])
  
  signac_object[[i]] <- CreateSeuratObject(assay, 
                                           assay = "peaks", 
                                           meta.data=object$md[[i]])
  signac_object[[i]]$section <- rownames(infoTable)[i]
  # signac_object[[i]]$sample <- i
  signac_object[[i]]$sample <- rownames(infoTable)[i]
  # compute LSI
  signac_object[[i]] <- FindTopFeatures(signac_object[[i]], min.cutoff = 10)
  signac_object[[i]] <- RunTFIDF(signac_object[[i]])
  signac_object[[i]] <- RunSVD(signac_object[[i]])
}

Warning message in CreateChromatinAssay(object$mtx[[i]], fragments = object$frag[[i]]):
"Overlapping ranges supplied. Ranges should be non-overlapping."
Performing TF-IDF normalization

Warning message in RunTFIDF.default(object = GetAssayData(object = object, layer = "counts"), :
"Some features contain 0 total counts"
Running SVD

Scaling cell embeddings

Warning message in CreateChromatinAssay(object$mtx[[i]], fragments = object$frag[[i]]):
"Overlapping ranges supplied. Ranges should be non-overlapping."
Performing TF-IDF normalization

Warning message in RunTFIDF.default(object = GetAssayData(object = object, layer = "counts"), :
"Some features contain 0 total counts"
Running SVD

Scaling cell embeddings

Warning message in CreateChromatinAssay(object$mtx[[i]], fragments = object$frag[[i]]):
"Overlapping ranges supplied. Ranges should be non-overlapping."
Performing TF-IDF normalization

Warning message in RunTFIDF.default(object = GetAssayData(object = object, layer = "counts"), :


In [73]:
# merge objects
combined <- merge(
  x = signac_object[[1]],
  y = c(signac_object[2:length(dirs)])
)

table(combined$section)

# add spatial data to meta
for(i in seq_along(infoTable$tissue_paths)){
  tissue_md[[i]]$barcode <- paste0(tissue_md[[i]]$barcode, "_", i)
  rownames(tissue_md[[i]]) <- tissue_md[[i]]$barcode 
}

tissue_md_combined <- do.call("rbind", tissue_md) 
combined <- AddMetaData(combined, tissue_md_combined)


Warning message:
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


Warning message in CreateChromatinAssay(counts = merged.counts, min.cells = -1, :
"Overlapping ranges supplied. Ranges should be non-overlapping."



220327_C1 220327_C2 220327_D2 
     3538      2795      3533 

Don't add the image as mine has already been processed.

In [76]:
infoTable$spotfiles

$`220327_C1`
[1] "../data/spatial_atac/github_folder/meta/220327_C1_tissue.csv"

$`220327_C2`
[1] "../data/spatial_atac/github_folder/meta/220327_C2_tissue.csv"

$`220327_D2`
[1] "../data/spatial_atac/github_folder/meta/220327_D2_tissue.csv"

In [75]:
# Extract Only `220327_C1`
signac_filtered <- subset(combined, subset = sample == "220327_C1")

# Save Processed Object
saveRDS(signac_filtered, "220327_C1_signac_object.rds")

# Print Summary
print(signac_filtered)

An object of class Seurat 
215978 features across 3538 samples within 1 assay 
Active assay: peaks (215978 features, 0 variable features)
 2 layers present: counts, data


In [79]:
#preprocess raw data
signac_filtered <- signac_filtered %>% 
  RunTFIDF() %>%
  FindTopFeatures(min.cutoff = 'q0') %>%
  RunSVD() %>%
  RunUMAP(reduction = 'lsi', dims = 2:7) %>%
  FindNeighbors(reduction = 'lsi', dims = 2:7) %>%
  FindClusters(algorithm = 3, resolution = 0.5)


Performing TF-IDF normalization

Warning message in RunTFIDF.default(object = GetAssayData(object = object, layer = "counts"), :
"Some features contain 0 total counts"


Running SVD

Scaling cell embeddings

15:39:46 UMAP embedding parameters a = 0.9922 b = 1.112

15:39:46 Read 3538 rows and found 6 numeric columns

15:39:46 Using Annoy for neighbor search, n_neighbors = 30

15:39:46 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

15:39:47 Writing NN index file to temp file /tmp/RtmplafeTH/file12eabe5e06dc39

15:39:47 Searching Annoy index using 1 thread, search_k = 3000

15:39:48 Annoy recall = 100%

15:39:51 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

15:39:55 Initializing from normalized Laplacian + noise (using RSpectra)

15:39:55 Commencing optimization for 500 epochs, with 115044 positive edges

15:39:55 Using rng type: pcg

15:40:01 Optimization finished

Computing nearest neighbor gr

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 3538
Number of edges: 99972

Running smart local moving algorithm...
Maximum modularity in 10 random starts: 0.6749
Number of communities: 5
Elapsed time: 2 seconds


In [80]:
#save matrix for denoising
write.csv(signac_filtered@assays$peaks@data,
          "signac_filtered_brca_q0_peak_bc_matrix.csv")

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 5.7 GiB"


In [81]:
head(signac_filtered@assays$peaks@data)

  [[ suppressing 3538 column names 'AAACAAGGGATCAAAT-1_1', 'AAACAGCAGTCTGCTA-1_1', 'AAACATTCGGGATTCT-1_1' ... ]]



6 x 3538 sparse Matrix of class "dgCMatrix"
                                                                               
chr1-975451-975952   . . . . . . . . . . . . . . .        . . . . . . . . . . .
chr1-1014228-1014729 . . . . . . . . . . . . . . 6.896184 . . . . . . . . . . .
chr1-1290080-1290581 . . . . . . . . . . . . . . .        . . . . . . . . . . .
chr1-1291099-1291600 . . . . . . . . . . . . . . .        . . . . . . . . . . .
chr1-1291742-1292243 . . . . . . . . . . . . . . .        . . . . . . . . . . .
chr1-1327977-1328478 . . . . . . . . . . . . . . .        . . . . . . . . . . .
                                                                              
chr1-975451-975952   . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-1014228-1014729 . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-1290080-1290581 . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1-1291099-1291600 . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
c

# DCA - Peaks denoiser

On terminal, in a Python environment with TensorFlow, such as `anndatas` run:
```sh
dca signac_filtered_brca_q0_peak_bc_matrix.csv output/sATAC_preprocessing/dca_peaks_q0_brca \
 --threads 3 \
 --nosizefactors --nonorminput --nologinput \
 --saveweights
```

---

In [77]:
PATH_TO_SAMPLE_NEW_POSITION_LIST <- "../4_clustering_and_classification/output/UNI2-h/satac_C1_v3_allspots_&_visium_FFPE_dcis_idc_10X_img_not_changed_allspots/on_individual_samples/tissue_position_list_with_6_kmeans_clusters_visium_FFPE_dcis_idc_10X_&_img_not_changed_allspots_&_target_is_reference_full_100um_fromWSI_histomicsTK_macenko_nomasking.csv"

In [ ]:
infoTable$spotfiles <- paste0(strsplit(infoTable$spotfiles, ".csv"), "_positions_list.csv")
se <- InputFromTable(infoTable, scaleVisium = 1)
se <- LoadImages(se, time.resolve = F)
signac_filtered@tools[["Staffli"]] <- se@tools$Staffli # create STutility object for spatial plots

In [24]:
Assays(combined)

An object of class "SimpleAssays"
Slot "data":
List of length 1


In [ ]:
peak_counts <- combined@assays$peaks@counts
peak_counts

In [ ]:
normalized_data <- combined@assays$peaks@data
normalized_data

In [ ]:
peak_accessibility <- FetchData(combined, vars = "chr1-975451-975952")
peak_accessibility

In [46]:
scaled_data <- combined@assays$peaks@scale.data
scaled_data

<0 x 0 matrix>

In [ ]:
# BiocManager::install("EnsDb.Hsapiens.v86")
# BiocManager::install("BSgenome.Hsapiens.UCSC.hg38")
# devtools::install_github("jbergenstrahle/STUtility")
# remotes::install_github('satijalab/seurat-wrappers')
# devtools::install_github('cole-trapnell-lab/monocle3')
# devtools::install_github("GreenleafLab/ArchR", ref="master", repos = BiocManager::repositories())

In [26]:
combined

An object of class Seurat 
215978 features across 9866 samples within 1 assay 
Active assay: peaks (215978 features, 0 variable features)
 2 layers present: counts, data

In [29]:
combined[["peaks"]]

ChromatinAssay data with 215978 features for 9866 cells
Variable features: 0 
Genome: 
Annotation present: FALSE 
Motifs present: FALSE 
Fragment files: 3 

In [34]:
combined[["peaks"]]["annotation"]

NULL

In [ ]:
peak_counts <- layer(combined)

In [ ]:
combined$data

In [21]:
head(combined@meta.data)

,orig.ident,nCount_peaks,nFeature_peaks,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,nonprimary,⋯,peak_region_fragments,peak_region_cutsites,section,sample,barcode,tissue,y,x,pixel_y,pixel_x
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>
AAACAAGGGATCAAAT-1_1,SeuratProject,1639,1602,45222,19934,127,445,5367,12977,17,⋯,1033,1896,220327_C1,1,AAACAAGGGATCAAAT-1_1,1,32,72,8425,32857
AAACAGCAGTCTGCTA-1_1,SeuratProject,1282,1256,31808,15667,76,239,3535,7173,13,⋯,795,1485,220327_C1,1,AAACAGCAGTCTGCTA-1_1,1,47,33,6475,34162
AAACATTCGGGATTCT-1_1,SeuratProject,1360,1346,32228,15483,54,334,3816,7440,15,⋯,883,1630,220327_C1,1,AAACATTCGGGATTCT-1_1,1,47,23,5975,34162
AAACCTGGCACACCCT-1_1,SeuratProject,2024,1975,51321,25784,66,442,5787,10593,56,⋯,1239,2313,220327_C1,1,AAACCTGGCACACCCT-1_1,1,22,34,6525,31987
AAACGCCGCGTTTCGA-1_1,SeuratProject,240,240,5814,2589,21,56,657,1587,1,⋯,139,260,220327_C1,1,AAACGCCGCGTTTCGA-1_1,1,25,93,9475,32248
AAACGCGGCGAAAGTA-1_1,SeuratProject,617,610,15876,7621,34,144,1964,3731,10,⋯,385,703,220327_C1,1,AAACGCGGCGAAAGTA-1_1,1,39,91,9375,33466


In [ ]:
# # Set Paths
# PATH_TO_sATAC_DATA_FOLDER <- "../data/spatial_atac/"
# SAMPLE_NAME <- "220327_C1"

# # Create Paths for the Sample
# infoTable <- data.frame(
#   singlecell = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", SAMPLE_NAME, "/singlecell.csv"),
#   fragments = paste0(PATH_TO_sATAC_DATA_FOLDER, "outs/", SAMPLE_NAME, "/fragments.tsv.gz"),
#   tissue_paths = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", SAMPLE_NAME, "_tissue.tsv"),
#   spotfiles = paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/", SAMPLE_NAME, "_tissue.csv"),
#   stringsAsFactors = FALSE
# )

# # Load Metadata
# if (!file.exists(infoTable$singlecell)) stop("File not found: ", infoTable$singlecell)
# md <- read.table(infoTable$singlecell, sep = ",", header = TRUE, row.names = 1)[-1, ]  # Remove NO_BARCODE row

# # Create Fragment Object
# if (!file.exists(infoTable$fragments)) stop("File not found: ", infoTable$fragments)
# frag_obj <- CreateFragmentObject(path = infoTable$fragments, cells = rownames(md))

# # Load Peak Counts (from your .h5 file)
# counts_matrix <- Read10X_h5(paste0(PATH_TO_sATAC_DATA_FOLDER, "combined/counts_brca_satac.h5"))

# # Ensure Only the Current Sample's Data is Used
# counts_matrix <- counts_matrix[, colnames(counts_matrix) %in% rownames(md)]

# # Create Chromatin Assay
# assay <- CreateChromatinAssay(counts_matrix, fragments = frag_obj)

# # Create Signac Object
# signac_object <- CreateSeuratObject(assay, assay = "peaks", meta.data = md)

# # Add Sample Name
# signac_object$section <- SAMPLE_NAME
# signac_object$sample <- SAMPLE_NAME

# # Normalize and Reduce Dimensions
# signac_object <- FindTopFeatures(signac_object, min.cutoff = 10)
# signac_object <- RunTFIDF(signac_object)
# signac_object <- RunSVD(signac_object)

# # Save Processed Object
# saveRDS(signac_object, paste0(SAMPLE_NAME, "_signac_object.rds"))

# # Print Summary
# print(signac_object)


Computing hash

Warning message in CreateChromatinAssay(counts_matrix, fragments = frag_obj):
"Overlapping ranges supplied. Ranges should be non-overlapping."


ERROR: Error in CreateChromatinAssay(counts_matrix, fragments = frag_obj): No cells retained due to minimum feature cutoff supplied


In [12]:
# BRCA_PEAK_CALLS <- paste0(PATH_TO_sATAC_DATA_FOLDER, "github_folder/meta/BRCA_peakCalls.csv")

# gr <- read.csv(
#   file = BRCA_PEAK_CALLS,
#   col.names = c("chr", "start", "end"),
#   sep = ";"
# ) %>% makeGRangesFromDataFrame()

In [13]:
# gr[0:5]

In [14]:
# # create fragment objects and count matrices for each section separately
# i <- sATAC_SAMPLE_NAME
# i

# folder = paste0(PATH_TO_sATAC_DATA_FOLDER, "combined/")
# # load metadata
# object$md[[i]] <- read.table(
# file = infoTable$singlecell[i],
# stringsAsFactors = FALSE,
# sep = ",",
# header = TRUE,
# row.names = 1
# )[-1, ] # remove the first row (NO_BARCODE)

# # create fragment objects
# object$frag[[i]] <- CreateFragmentObject(
# path = infoTable$fragments[i],
# cells = rownames(object$md[[i]])
# )

# if (file.exists(paste0(folder, "brca_peak_bc_matrix.h5"))) {
# message("File exists.")
# object$counts[[i]] <- Read10X_h5(paste0(folder, "brca_peak_bc_matrix.h5"))

# } else {

# # make count matrix
# object$counts[[i]] <- FeatureMatrix(
#     fragments = object$frag[[i]],
#     features = gr,
#     cells = rownames(object$md[[i]])
# )

# write10xCounts(path = paste0(folder, "brca_peak_bc_matrix.h5"),
#                 x = object$counts[[i]],
#                 type = "HDF5") # save new count matrix
# }


In [15]:
# # create fragment objects and count matrices for each section separately
# for(i in seq_along(list(sATAC_SAMPLE_NAME))){
#   folder = paste0(PATH_TO_sATAC_DATA_FOLDER, "combined/")
#   # load metadata
#   object$md[[i]] <- read.table(
#     file = infoTable$singlecell[i],
#     stringsAsFactors = FALSE,
#     sep = ",",
#     header = TRUE,
#     row.names = 1
#   )[-1, ] # remove the first row (NO_BARCODE)
  
#   # create fragment objects
#   object$frag[[i]] <- CreateFragmentObject(
#     path = infoTable$fragments[i],
#     cells = rownames(object$md[[i]])
#   )
  
#   if (file.exists(paste0(folder, "brca_peak_bc_matrix.h5"))) {
#     message("File exists.")
#     object$counts[[i]] <- Read10X_h5(paste0(folder, "brca_peak_bc_matrix.h5"))
    
#   } else {
    
#     # make count matrix
#     object$counts[[i]] <- FeatureMatrix(
#       fragments = object$frag[[i]],
#       features = gr,
#       cells = rownames(object$md[[i]])
#     )
    
#     write10xCounts(path = paste0(folder, "brca_peak_bc_matrix.h5"),
#                    x = object$counts[[i]],
#                    type = "HDF5") # save new count matrix
#   }
  
# }